## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [0]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
project_path = "/content/drive/My Drive/cnn_project_dog_breed/"

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'train.zip', 'r') as z:
  z.extractall()

Repeat the same step for test.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'test.zip', 'r') as z:
  z.extractall()

Repeat the same step for sample_submission.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'sample_submission.csv.zip', 'r') as z:
  z.extractall()

Repeat the same step for labels.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'labels.csv.zip', 'r') as z:
  z.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [0]:
import pandas as pd
labels = pd.read_csv('/content/labels.csv')

In [6]:
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


### Print the count of each category of Dogs given in the dataset



In [7]:
labels['breed'].value_counts()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
                       ... 
golden_retriever         67
brabancon_griffon        67
komondor                 67
briard                   66
eskimo_dog               66
Name: breed, Length: 120, dtype: int64

In [12]:
labels['breed'].shape

(10222,)

### Get one-hot encodings of labels

In [0]:
one_hot_labels = pd.get_dummies(labels['breed'])

In [15]:
one_hot_labels.head()

,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,bedlington_terrier,bernese_mountain_dog,black-and-tan_coonhound,blenheim_spaniel,bloodhound,bluetick,border_collie,border_terrier,borzoi,boston_bull,bouvier_des_flandres,boxer,brabancon_griffon,briard,brittany_spaniel,bull_mastiff,cairn,cardigan,chesapeake_bay_retriever,chihuahua,chow,clumber,cocker_spaniel,collie,curly-coated_retriever,dandie_dinmont,dhole,dingo,doberman,english_foxhound,...,norwegian_elkhound,norwich_terrier,old_english_sheepdog,otterhound,papillon,pekinese,pembroke,pomeranian,pug,redbone,rhodesian_ridgeback,rottweiler,saint_bernard,saluki,samoyed,schipperke,scotch_terrier,scottish_deerhound,sealyham_terrier,shetland_sheepdog,shih-tzu,siberian_husky,silky_terrier,soft-coated_wheaten_terrier,staffordshire_bullterrier,standard_poodle,standard_schnauzer,sussex_spaniel,tibetan_mastiff,tibetan_terrier,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [6]:
from tqdm import tqdm
import cv2

x_train = []
y_train = []


for a,b in tqdm(labels.values):
  img = cv2.imread('/content/train/{}.jpg'.format(a))
  img_resize = cv2.resize(img, (128,128))
  x_train.append(img_resize)
  y_train.append(b)

100%|██████████| 10222/10222 [00:30<00:00, 336.00it/s]


In [17]:
y_train[0]

'boston_bull'

In [18]:
x_train[0].shape

(128, 128, 3)

Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [0]:
import numpy as np

x_train_data = np.array([img for img in x_train])


In [10]:
x_train_data.shape

(10222, 128, 128, 3)

In [0]:
# y_train_data = np.array([data for data in y_train])
import pandas as pd
y_train_data = pd.get_dummies(y_train)

In [22]:
y_train_data.shape

(10222, 120)

In [0]:
type(y_train_data)

pandas.core.frame.DataFrame

In [9]:
import numpy as np
y_train_data = np.array(y_train_data)
type(y_train_data)

numpy.ndarray

In [0]:
i

0

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [10]:
from sklearn.model_selection import train_test_split
x_train2, x_val, y_train2, y_val = train_test_split(x_train_data, y_train_data, test_size=0.3, random_state=10)
print (len(x_train2))
print (len(x_val))

7155
3067


In [0]:
y_val.shape

(3067, 120)

### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [0]:
data_sample = pd.read_csv('/content/sample_submission.csv')

In [0]:
test_img = data_sample['id']

In [16]:
test_img

0        000621fb3cbb32d8935728e48679680e
1        00102ee9d8eb90812350685311fe5890
2        0012a730dfa437f5f3613fb75efcd4ce
3        001510bc8570bbeee98c8d80c8a95ec1
4        001a5f3114548acdefa3d4da05474c2e
                       ...               
10352    ffeda8623d4eee33c6d1156a2ecbfcf8
10353    fff1ec9e6e413275984966f745a313b0
10354    fff74b59b758bbbf13a5793182a9bbe4
10355    fff7d50d848e8014ac1e9172dc6762a3
10356    fffbff22c1f51e3dc80c4bf04089545b
Name: id, Length: 10357, dtype: object

Run the below code to load the test image files in x_test_feature

In [28]:
x_test_feature = []
i = 0 # initialisation
for f in tqdm(test_img.values): # f for format ,jpg
    img = cv2.imread('/content/test/{}.jpg'.format(f), 0)
    img_resize = cv2.resize(img, (128, 128)) 
    x_test_feature.append(img_resize)

100%|██████████| 10357/10357 [00:12<00:00, 799.99it/s]


Normalize the test data and convert it into 4 dimensions

In [0]:
x_test_feature_data = np.array([img for img in x_test_feature])


In [30]:
x_test_feature_data.shape

(10357, 128, 128)

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [0]:
from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense

from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import adam

In [0]:
model = Sequential()
model.add(Convolution2D(filters=64, kernel_size=5, activation ='relu', input_shape = (128, 128, 3))) 

model.add(Convolution2D(filters=64, kernel_size=3, padding='same', activation='relu'))

model.add(Flatten()) 

model.add(Dense(units=256,activation = 'relu'))

model.add(Dense(units = 120, activation = 'softmax')) 


In [13]:
optimizer = adam(lr=0.001)
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 124, 124, 64)      4864      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 124, 124, 64)      36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 984064)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               251920640 
_________________________________________________________________
dense_2 (Dense)              (None, 120)               30840     
Total params: 251,993,272
Trainable params: 251,993,272
Non-trainable params: 0
_________________________________________________________________


### Use batch_size = 128 and epochs = 10 and execute the model

In [15]:
history = model.fit(x_train2,y_train2,
                    epochs=10, 
                    batch_size = 128,
                    validation_data=(x_val,y_val),
                    verbose = 1,
                    initial_epoch=0)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7155 samples, validate on 3067 samples
Epoch 1/10
7155/7155 [==============================] - 25s 4ms/step - loss: 15.9921 - acc: 0.0077 - val_loss: 15.9604 - val_acc: 0.0098
Epoch 2/10
7155/7155 [==============================] - 14s 2ms/step - loss: 15.9919 - acc: 0.0078 - val_loss: 15.9604 - val_acc: 0.0098
Epoch 3/10
7155/7155 [==============================] - 14s 2ms/step - loss: 15.9919 - acc: 0.0078 - val_loss: 15.9604 - val_acc: 0.0098
Epoch 4/10
7155/7155 [==============================] - 14s 2ms/step - loss: 15.9919 - acc: 0.0078 - val_loss: 15.9604 - val_acc: 0.0098
Epoch 5/10
7155/7155 [==============================] - 14s 2ms/step - loss: 15.9919 - acc: 0.0078 - val_loss: 15.9604 - val_acc: 0.0098
Epoch 6/10
7155/7155 [==============================] - 14s 2ms/step - loss: 15.9919 - acc: 0.0078 - val_loss: 15.9604 - val_acc: 0.0098
Epoch 7/10
7155/7155 [============

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [0]:
from keras.preprocessing.image import ImageDataGenerator


In [0]:
datagen = ImageDataGenerator(horizontal_flip=True, rotation_range=90, brightness_range=[0.2,1.0])


In [0]:
datagen

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [0]:
train_generator = datagen.flow(x_train2, y_train2, batch_size=128)

In [0]:
val_generator = datagen.flow(x_val,y_val, batch_size=128)

In [0]:
x_train2.shape

(7155, 128, 128, 3)

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [20]:
history = model.fit_generator(train_generator,
                    epochs=10,                
                    validation_data=val_generator,
                    initial_epoch=0)

Epoch 1/10
56/56 [==============================] - 48s 858ms/step - loss: 15.9922 - acc: 0.0078 - val_loss: 15.9604 - val_acc: 0.0098
Epoch 2/10
56/56 [==============================] - 42s 742ms/step - loss: 15.9919 - acc: 0.0078 - val_loss: 15.9604 - val_acc: 0.0098
Epoch 3/10
56/56 [==============================] - 42s 746ms/step - loss: 15.9922 - acc: 0.0078 - val_loss: 15.9604 - val_acc: 0.0098
Epoch 4/10
56/56 [==============================] - 41s 740ms/step - loss: 15.9922 - acc: 0.0078 - val_loss: 15.9604 - val_acc: 0.0098
Epoch 5/10
56/56 [==============================] - 42s 742ms/step - loss: 15.9919 - acc: 0.0078 - val_loss: 15.9604 - val_acc: 0.0098
Epoch 6/10
56/56 [==============================] - 41s 739ms/step - loss: 15.9922 - acc: 0.0078 - val_loss: 15.9604 - val_acc: 0.0098
Epoch 7/10
56/56 [==============================] - 42s 743ms/step - loss: 15.9922 - acc: 0.0078 - val_loss: 15.9604 - val_acc: 0.0098
Epoch 8/10
56/56 [==============================] - 42s

# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [0]:
from keras.layers import Dropout, Flatten, Dense, Input

from keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
# input_tensor = Input(shape=(128, 128, 3))

base_model= VGG16(weights=(project_path+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg',input_shape=(128,128,3))

Print the summary of the base_model

In [110]:

base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [0]:
base_model_1 = Sequential()
base_model_1.add(base_model)

# base_model_1.add(Flatten()) 

base_model_1.add(Dense(units=1024,activation = 'relu'))
base_model_1.add(Dense(units=256,activation = 'relu'))


base_model_1.add(Dense(units = 120, activation = 'softmax')) 




### Make all the layers in the base_model (VGG16) to be non-trainable

In [115]:
#Freezing layers in the model which don't have 'dense' in their name
for layer in base_model_1.layers:
  if('dense' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in model.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

conv2d_1
True
conv2d_2
True
flatten_1
True
dense_1
True
dense_2
True


### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [0]:
ptimizer = adam(lr=0.001)
base_model_1.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [0]:
# base_model_1.summary()

In [117]:
history = base_model_1.fit(x_train2,y_train2,
                    epochs=1, 
                    batch_size = 128,
                    validation_data=(x_val,y_val),
                    verbose = 1,
                    initial_epoch=0)

Train on 7155 samples, validate on 3067 samples
Epoch 1/1
7155/7155 [==============================] - 12s 2ms/step - loss: 5.7426 - acc: 0.0520 - val_loss: 4.1520 - val_acc: 0.1112


In [120]:
history = base_model_1.fit_generator(train_generator,
                    epochs=10,                
                    validation_data=val_generator,
                    initial_epoch=0)

Epoch 1/10
56/56 [==============================] - 47s 840ms/step - loss: 3.4820 - acc: 0.1998 - val_loss: 3.5725 - val_acc: 0.1836
Epoch 2/10
56/56 [==============================] - 41s 730ms/step - loss: 3.1041 - acc: 0.2711 - val_loss: 3.3309 - val_acc: 0.2240
Epoch 3/10
56/56 [==============================] - 41s 737ms/step - loss: 2.8487 - acc: 0.3190 - val_loss: 3.2948 - val_acc: 0.2338
Epoch 4/10
56/56 [==============================] - 41s 732ms/step - loss: 2.7178 - acc: 0.3393 - val_loss: 3.2059 - val_acc: 0.2452
Epoch 5/10
56/56 [==============================] - 41s 741ms/step - loss: 2.5734 - acc: 0.3707 - val_loss: 3.1428 - val_acc: 0.2638
Epoch 6/10
56/56 [==============================] - 41s 731ms/step - loss: 2.4455 - acc: 0.3976 - val_loss: 3.2048 - val_acc: 0.2723
Epoch 7/10
56/56 [==============================] - 41s 737ms/step - loss: 2.3300 - acc: 0.4192 - val_loss: 3.1822 - val_acc: 0.2638
Epoch 8/10
56/56 [==============================] - 41s 738ms/step - 

In [118]:
base_model_1.summary()

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_33 (Dense)             (None, 1024)              525312    
_________________________________________________________________
dense_34 (Dense)             (None, 256)               262400    
_________________________________________________________________
dense_35 (Dense)             (None, 120)               30840     
Total params: 15,533,240
Trainable params: 818,552
Non-trainable params: 14,714,688
_________________________________________________________________
